In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import glob
import imageio

In [20]:
def processData(imPath, labelPath):
    df = pd.read_csv('./train/actions.csv', header=None)
    yEncoder = OneHotEncoder(handle_unknown='ignore')
    labels = yEncoder.fit_transform(df).toarray()
    #yEncoder.fit(df)
    #labelNames = yEncoder.get_feature_names()
    data = []
    imPaths = glob.glob(imPath)
    for i in imPaths:
        ID = i.split('\\')[1].split('.')[0]
        tmp = imageio.imread(i)
        tmp = [tmp/255.0, labels[int(ID)]]
        #tmp = [tmp/255.0, df.loc[int(ID)].values[0]]
        data.append(tmp)
    return data

In [21]:
data_pair = processData('./train/train image/*.png', './train/actions.csv')

In [22]:
train, test = train_test_split(data_pair, test_size=0.2)
trainX = np.array([i[0] for i in train])
trainY = np.array([i[1] for i in train])
testX = np.array([i[0] for i in test])
testY = np.array([i[1] for i in test])
print(trainX.shape)
print(trainY.shape)

(13040, 64, 64, 3)
(13040, 12)


In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 128)      

In [ ]:
model.fit(trainX, trainY, batch_size=200, epochs=30, verbose=1)

Epoch 1/30
66/66 [==============================] - 45s 680ms/step - loss: 2.7111 - accuracy: 0.0885
Epoch 2/30
66/66 [==============================] - 44s 671ms/step - loss: 2.4866 - accuracy: 0.0840
Epoch 3/30
66/66 [==============================] - 44s 663ms/step - loss: 2.4848 - accuracy: 0.0862
Epoch 4/30
66/66 [==============================] - 43s 653ms/step - loss: 2.4845 - accuracy: 0.0863
Epoch 5/30
66/66 [==============================] - 43s 647ms/step - loss: 2.4847 - accuracy: 0.0898
Epoch 6/30
66/66 [==============================] - 43s 655ms/step - loss: 2.4843 - accuracy: 0.0876
Epoch 7/30
66/66 [==============================] - 44s 666ms/step - loss: 2.4846 - accuracy: 0.0884
Epoch 8/30
66/66 [==============================] - 42s 638ms/step - loss: 2.4847 - accuracy: 0.0885
Epoch 9/30
66/66 [==============================] - 45s 689ms/step - loss: 2.4841 - accuracy: 0.0881
Epoch 10/30
66/66 [==============================] - 43s 648ms/step - loss: 2.4845 - accura

In [ ]:
model.evaluate(testX,  testY)